In [11]:
from pywinauto import Application

### backend란? 

pywinauto를 설치하고나면 해야하는 것은 어떤 어플리케이션에
 접근하는 기술(pywinauto 백엔드)를 사용할 지를 정해야 한다.

- Win32 API(backend="win32") 현재 디폴트값

MFC, VB6, VCL,  간단한 WinForms과 대부분 레거시 앱에 대한 통제가 필요할 때 사용됨.

- MS UI Automation

Winforms, WPF, 스토어앱들, Qt5, 브라우저(크롬의 경우에는 —force-renderer-accessibility 라는 플래그를 실행전에 켜주어야 한다.

이 예제에서는 kakao talk을 다룰 것 이기 때문에 backend로 uia(MS UI Automation)를 사용할 것이다. 만일 backend로 win32 api를 이용할 것이라면 backend='win32'를 지정해주면된다.

### pywinauto는 2가지 방법으로 window application을 제어할 수 있다.
- start
제어하려는 어플리케이션이 실행되지 않았을 때, 실행시키는 동시에 해당 어플리케이션을 Application object로 얻을 수 있.

- connect
이미 실행된 어플리케이션의 process id , title, file경로 등을 통해서 Application object를 얻을 수 있다.

In [47]:
kakao_application = Application(backend='uia').start(r"C:\Program Files (x86)\Kakao\KakaoTalk\KakaoTalk.exe")

### psutil module로 process id를 얻고 그 process id를 이용하여 Application객체를 만들 수 있다. 하지만, 동일한 이름의 process가 많은 경우에는 적합하지 않다. 

In [48]:
process_id = 0
process_handle = 0
import psutil
for process in psutil.process_iter():
    if "Kakao" in process.name():
        print(process.name())
        process_id = process.pid 
        

kakao_application = Application(backend='uia').connect(process=process_id)

KakaoTalk.exe


### Application 객체에서 WindowSpecification으로 변환하기
WindowSpecification 클래스는 pywinauto api의 핵심 개념이다. 
WindowSpecification은 매칭/탐색 알고리즘을 이용하여 실제 윈도우 객체를 얻을 수 있다.
Application객체에서 WindowSpecification객체로의 첫 진입점은 top_window메소드를 통해서 이루어진다.

In [49]:
top_window_object = kakao_application.top_window()

### application 구조 탐색하기 : print_control_identifiers
print_control_identifiers를 이용하여 application의 계층구조를 볼 수 있다. HTML처럼 계층구조임.

In [50]:
top_window_object.print_control_identifiers()

Control Identifiers:

Dialog - 'KakaoTalk'    (L1107, T134, R1831, B897)
['KakaoTalk', 'KakaoTalkDialog', 'Dialog']
child_window(title="KakaoTalk", control_type="Window")
   | 
   | Pane - 'KakaoTalkEdgeWnd'    (L1107, T134, R1831, B897)
   | ['Pane', 'KakaoTalkEdgeWndPane', 'KakaoTalkEdgeWnd', 'Pane0', 'Pane1']
   | child_window(title="KakaoTalkEdgeWnd", control_type="Pane")
   | 
   | Pane - 'OnlineMainView_0x000505f6'    (L1108, T165, R1830, B795)
   | ['Pane2', 'OnlineMainView_0x000505f6Pane', 'OnlineMainView_0x000505f6']
   | child_window(title="OnlineMainView_0x000505f6", auto_id="132", control_type="Pane")
   |    | 
   |    | Pane - 'ContactListView_0x000700bc'    (L1108, T227, R1830, B795)
   |    | ['Pane3', 'ContactListView_0x000700bcPane', 'ContactListView_0x000700bc']
   |    | child_window(title="ContactListView_0x000700bc", auto_id="130", control_type="Pane")
   |    |    | 
   |    |    | Pane - 'ContactListCtrl_0x00020352'    (L1108, T270, R1830, B795)
   |    |    | [

가장 최상위에 Dialog control type이며, 이름은 KakaoTalk인 windowSpecification 객체로 이루어져있고 이 최상위 객체는 자식으로 3개의 Pane WindowSpecification객체를 가지고 있다. 자식으로 이동하기 위해서는 

   | Pane - 'KakaoTalkEdgeWnd'    (L1107, T134, R1831, B897)
   | ['Pane', 'KakaoTalkEdgeWndPane', 'KakaoTalkEdgeWnd', 'Pane0', 'Pane1']
   | child_window(title="KakaoTalkEdgeWnd", control_type="Pane")

top_window_object.child_window(title="KakaoTalkEdge\nd", control_type="Pane") 

top_window_object.child_window(title_re="KakaoTalkEdge\nd", control_type="Pane") # title_re 는 regular expression을 이용하기임 

top_window_object.Pane 

top_window_object.KakaoTalkEdgeWndPane 

top_window_object.KakaoTalkEdgeWnd 

top_window_object.Pane0

top_window_object.Pane1 

top_window_object["Pane1"] 

top_window_object["Pane0"]

top_window_object["KakaoTalkEdgeWnd"]

top_window_object.window(title="KakaoTalkEdgeWndPane")

등이다. title이 명확하지 않은 경우에는 title_re를 통해서 regular expression 방식으로 찾을 수 있다. 혹은
['Pane', 'KakaoTalkEdgeWndPane', 'KakaoTalkEdgeWnd', 'Pane0', 'Pane1'] 이러한 리스트중 하나를 골라서 attribute에 접근하듯 접근자 . 을 이용하여 접근 할 수도 있으며 dictionary처럼 ["attribute이름"]을 통해 접근할 수도 있다. 

### 각 컨트롤 타입마다 할 수 있는 행위들이 지정되어 있음. control_type을 본 후  https://pywinauto.readthedocs.io/en/latest/controls_overview.html 에서 참고하여 제어할 것